<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/Q%26A_ChatBot2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 인공지능 PDF Q&A 챗봇 프로젝트 2-5 : 1학기 최종본
# 추가점 1 : 프롬프트 메시지 변경 (완)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-dotenv

In [ ]:
!pip install langchain_openai==0.3.7

In [ ]:
!pip install langchain-huggingface==0.1.2

In [ ]:
!pip install langchain_community==0.3.18

In [ ]:
#!pip install faiss-cpu==1.10.0
!pip install langchain_chroma==0.2.2

In [ ]:
!pip install pypdf

In [ ]:
#!pip install pydantic==2.10.6
!pip uninstall -y gradio
!pip install --upgrade gradio gradio-client

In [ ]:
!pip uninstall numpy -y
!pip install --no-cache-dir numpy==1.26.4
# colab에 numpy 2.버전이 설치되어 있어서 버전 충돌남

In [1]:
import numpy
print(numpy.__version__)

1.26.4


In [2]:
import gradio
print(gradio.__version__)

5.33.0


In [ ]:
%pip install rank_bm25

In [ ]:
!pip install google-cloud-vision pdf2image
!apt-get install -y poppler-utils

In [ ]:
import gradio as gr
import os
import uuid
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.documents import Document
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

# 환경 변수 불러오기(openai API 키)
load_dotenv('/content/drive/MyDrive/Colab Notebooks/.env')

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini")

# 텍스트 분리
text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

# 임베딩 모델
hf_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 프롬프트 템플릿
system_message = """
당신은 공주대학교에 관한 질문에 응답하는 AI입니다. 모든 응답은 반드시 한국어로 작성되어야 하며, 벡터 DB에 저장된 공주대학교의 공식 문서만을 근거로 응답해야 합니다.

응답 시 다음 지침을 따르세요:

1. 질문에 명시된 부서 또는 사용자가 선택한 부서를 기준으로만 답변하세요.
   - 'ALL(전체)'인 경우, 질문 내용을 분석해 가장 관련 있는 부서를 우선 판단해 응답하세요.
   - 이후 대화에서 부서가 정정되면, 해당 부서를 기준으로 다시 검색해 답변하세요.

2. 검색된 문서가 여러 개인 경우, 다음 우선순위를 지키세요:
   - 파일명 또는 내용에 포함된 **연도 기준** 최신 문서 우선
   - 그 외엔 **작성일 또는 내용 신뢰도** 기준으로 판단

3. 관련 문서가 없거나 질문과 무관한 경우, 다음 문장을 그대로 출력하세요:
   “해당 질문에 대한 공식 문서나 파일이 없습니다.”

4. 인사, 감사, 사용법 등 일상적 질문은 자연스럽게 응답하되, 출처는 생략하거나 “해당 정보 없음”으로 표기하세요.

5. 모든 응답 하단에 다음 형식으로 출처를 표기하세요:
출처:
  - 파일명: ○○○ 또는 “해당 정보 없음”
  - 부서/학과: ○○○ 또는 “해당 정보 없음”
  - URL: ○○○ 또는 “해당 정보 없음”

※ “None”, 빈 문자열, null 등은 절대 그대로 출력하지 마세요. 반드시 “해당 정보 없음”으로 대체하세요.

예시:
답변: ○○○
출처:
  - 파일명: ○○○
  - 부서/학과: ○○○
  - URL: ○○○
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("placeholder", "{memory}"),
        ("human", "선택된 부서: {selected_dept}\n\n질문: {input}\n\n🔍 검색된 문서:\n{context}")
    ]
)

# 출력 파서
parser = StrOutputParser()

# 트리머 설정
trimmer = trim_messages(
    max_tokens=3500,
    token_counter=llm,
    strategy="last",
    include_system=True,
    start_on="human"
)

DEPARTMENT_MAP = {
    # 모든 문서 다 때려박음
    "ALL(전체)" : "knu_chroma_db_all",

    # 대표 사이트
    "National Kongju University Korean Language Representative (국립공주대학교 국문 대표)": "knu_chroma_db_national_kongju_university_korean_language_representative",

    # 공과 대학 학과
    "Software Department (소프트웨어학과)": "knu_chroma_db_software",
    "Department of Computer Engineering (컴퓨터공학과)": "knu_chroma_db_computer",
    "Department of Artificial Intelligence (인공지능학부)": "knu_chroma_db_artificial_intelligence",
    "Department of Architecture (건축학과)": "knu_chroma_db_architecture",
    "Department of Chemical Engineering (화학공학부)": "knu_chroma_db_chemical_engineering",
    "Department of Design Convergence (디자인컨버전스학과)": "knu_chroma_db_design_convergence",
    "Department of Digital Convergence Mold Engineering (디지털융합금형공학과)": "knu_chroma_db_digital_convergence_mold_engineering",
    "Department of Electrical, Electronic and Control Engineering (전기전자제어공학부)": "knu_chroma_db_electrical_electronic_and_control_engineering",
    "Department of Electrical, Electronic and Control Engineering (전기전자제어공학부)/Control and Instrumentation Engineering Major (제어계측공학전공)": "knu_chroma_db_control_and_instrumentation_engineering_major",
    "Department of Electrical, Electronic and Control Engineering (전기전자제어공학부)/Electrical Engineering Major (전기공학전공)": "knu_chroma_db_electrical_engineering_major",
    "Department of Electrical, Electronic and Control Engineering (전기전자제어공학부)/Electronic Engineering Major (전자공학전공)": "knu_chroma_db_electronic_engineering_major",
    "Department of Electrical, Electronic and Control Engineering (전기전자제어공학부)/Semiconductor Information Engineering Major (반도체정보공학전공)": "knu_chroma_db_semiconductor_information_engineering_major",
    "Department of Environmental Engineering (환경공학과)": "knu_chroma_db_environmental_engineering",
    "Department of Future Automotive Engineering (미래자동차공학과)": "knu_chroma_db_future_automotive_engineering",
    "Department of Industrial Engineering (산업공학과)": "knu_chroma_db_industrial_engineering",
    "Department of Information and Communication Engineering (정보통신공학과)": "knu_chroma_db_information_and_communication_engineering",
    "Department of Intelligent Mobility Engineering (지능형모빌리티공학과)": "knu_chroma_db_intelligent_mobility_engineering",
    "Department of Mechanical and Automotive Engineering (기계자동차공학부)": "knu_chroma_db_mechanical_and_automotive_engineering",
    "Department of New Materials Engineering (신소재공학부)": "knu_chroma_db_new_materials_engineering",
    "Department of Photonics (광공학과)": "knu_chroma_db_photonics",
    "Department of Smart Information Technology Engineering (스마트정보기술공학과)": "knu_chroma_db_smart_information_technology_engineering",
    "Department of Smart Infrastructure Engineering (스마트인프라공학과)": "knu_chroma_db_smart_infrastructure_engineering",
    "Department of Urban and Transportation Engineering (도시·교통공학과)": "knu_chroma_db_urban_and_transportation_engineering",
    "Green Smart Architectural Engineering Department (그린스마트건축공학과)": "knu_chroma_db_green_smart_architectural_engineering",

    # 대학원
    "Kongju National University Computer & Software & IT Convergence (국립공주대학교 컴퓨터&소프트웨어&IT융합)": "knu_chroma_db_computer_software_it_convergence",
    "All Graduate Schools (전체 대학원)": "knu_chroma_db_all_graduate_schools",

    # 사업단
    "National Kongju University SW Centered University Business Group (국립공주대학교 SW중심대학사업단)": "knu_chroma_db_sw_centerd_university_business_group",
}

# 전역
retriever = None
rag_chain = None
history_store: dict[str, InMemoryChatMessageHistory] = {}

# 세션 초기화 함수
def init_session(session_id: str):
    if not session_id or not isinstance(session_id, str):
        raise ValueError("유효한 session_id가 필요합니다.")

    if session_id not in history_store:
        history_store[session_id] = InMemoryChatMessageHistory()

    # SystemMessage가 이미 포함되어 있는지 확인
    existing = history_store[session_id].messages
    if not any(msg.type == "system" and msg.content.strip() == system_message.strip() for msg in existing):
        history_store[session_id].add_message(SystemMessage(content=system_message.strip()))

# 세션 히스토리 함수
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    return history_store[session_id]

def load_all_docs_once():
    global all_docs
    if not all_docs:
        for dept, dir_name in DEPARTMENT_MAP.items():
            db = Chroma(persist_directory=f"/content/drive/MyDrive/ChromaDB/{dir_name}", embedding_function=hf_embeddings)
            res = db.get(include=["documents", "metadatas"])
            all_docs.extend([Document(page_content=d, metadata=m) for d, m in zip(res["documents"], res["metadatas"])])

# RAG 체인 로드
def rebuild_chain(selected_dept: str = None):
    global retriever, rag_chain

    # 사용자가 부서를 선택한 경우: 해당 부서 전용 DB만 불러와 retriever 구성(빠름)
    if selected_dept:
        db_path = f"/content/drive/MyDrive/ChromaDB/{DEPARTMENT_MAP[selected_dept]}"
        db = Chroma(persist_directory=db_path, embedding_function=hf_embeddings)
    else:
        # 부서를 선택하지 않은 경우: 모든 부서의 문서가 저장된 DB를 로드하여 retriever 구성(느림)
        db_path = f"/content/drive/MyDrive/ChromaDB/knu_chroma_db_all"
        db = Chroma(persist_directory=db_path, embedding_function=hf_embeddings)

    """
    벡터 거리 기반 의미 검색
    질문 -> MultiQuery(3개의 서브 질문)
    서브 질문 1 -> ChromaDB에서 mmr방식(다양하고도 유사한 문서를 찾는 방식)으로 검색하여 10개 후보 중 5개를 선택
    서브 질문 2 -> ChromaDB에서 mmr방식으로 검색하여 10개 후보 중 5개를 선택
    서브 질문 3 -> ChromaDB에서 mmr방식으로 검색하여 10개 후보 중 5개를 선택

    키워드 기반 재정렬 - Document 리스트 상태에서 작동함
    청크 N개를 대상으로 BM25(키워드(=질문에서 나온 단어)가 문서 안에 얼마나 잘 등장하는지 관련성을 수치화하는 방식)

    앙상블 -> top 3개 선택
    -> LLM context에 top3 청크 3개 삽입 -> 답변 생성
    """
    # 전체 문서 로딩 후 BM25 구성용 문서 리스트 확보
    results = db.get(include=["documents", "metadatas"])
    docs = [Document(page_content=doc, metadata=meta) for doc, meta in zip(results["documents"], results["metadatas"])]

    # Chroma + BM25 혼합 검색기 구성
    chroma_retriever = db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5, "fetch_k": 10, "lambda_mult": 0.5}
    )

    bm25_retriever = BM25Retriever.from_documents(docs)
    bm25_retriever.k = 3
    base_retriever = EnsembleRetriever(retrievers=[chroma_retriever, bm25_retriever])
    retriever = MultiQueryRetriever.from_llm(base_retriever, llm=llm)

    # 최종 RAG 체인
    rag_chain_core = {
        "memory": trimmer,          # trimmer 적용
        "context": retriever,
        "input": RunnablePassthrough(),
        "selected_dept": RunnablePassthrough()
    } | prompt_template | llm | parser

    # 최종 RAG 체인
    rag_chain = RunnableWithMessageHistory(
        rag_chain_core,
        get_session_history,
        input_messages_key="input",
        history_messages_key="memory"
    )

# 질문 처리
def answer_question(question: str, session_id: str, selected_dept: str) -> str:
    if not session_id:
        return "⚠️ 세션을 먼저 생성하거나 선택해주세요."

    init_session(session_id)

    # rag_chain의 입력 형식에 맞춰 dict 구성
    chain_input = {
        "memory": None,        # RunnableWithMessageHistory가 알아서 처리
        "context": None,       # retriever가 알아서 처리
        "input": question,
        "selected_dept": selected_dept if selected_dept else "ALL(전체)"
    }

    # RAG 체인 호출 → 답변 생성(자동으로 history에 기록됨)
    answer = rag_chain.invoke(
        chain_input,
        config={"configurable": {"session_id": session_id}}
    )
    return answer

# 여러개의 Chroma DB에 저장된 문서 목록 보여주는 함수 -> 애도 안씀
def show_stored_documents():
    file_names = set()

    try:
        for dept, dir_name in DEPARTMENT_MAP.items():
            db = Chroma(
                persist_directory=f"/content/drive/MyDrive/ChromaDB/{dir_name}",
                embedding_function=hf_embeddings
            )
            result = db.get(include=["metadatas"])
            metas = result.get("metadatas", [])
            for meta in metas:
                file_name = meta.get("file_name", "Unknown")
                department = meta.get("department", dept)
                file_names.add(f"{file_name} ({department})")
    except Exception as e:
        return f"❗ DB에서 메타데이터 로드 중 오류 발생: {e}"

    if not file_names:
        return "📂 문서가 없습니다."

    return "📚 저장된 문서 목록:\n" + "\n".join(f"• {f}" for f in sorted(file_names))

# 세션별로 저장된 대화 내역 보여주는 함수(Human/AI)
def show_history(session_id: str) -> str:
    init_session(session_id)
    msgs = get_session_history(session_id).messages

    if not msgs:
        return "⚠️ 대화 기록이 없습니다."

    output = []
    for msg in msgs:
        if msg.type == "system":
            output.append(f"📘 System: {msg.content.strip()}")
        elif msg.type == "human":
            output.append(f"🧑 Human: {msg.content.strip()}")
        elif msg.type == "ai":
            output.append(f"🤖 AI: {msg.content.strip()}")
    return "\n\n".join(output)

# 앱 시작 시 기본 세션 생성 및 db 로드
default_session_id = f"session_{uuid.uuid4().hex[:8]}"
init_session(default_session_id)
rebuild_chain("ALL(전체)")

# Gradio UI 설정
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # ── 사이드바: 세션 관리 ──
    with gr.Sidebar():
        session_dropdown = gr.Dropdown(
            label="채팅 세션 선택",
            choices=[default_session_id],
            value=default_session_id,
            interactive=True
        )
        new_session_btn = gr.Button("➕ 새 채팅")
        del_session_btn = gr.Button("🗑 세션 삭제")

        department_dropdown = gr.Dropdown(
            label="🔍 검색할 부서/학과 선택",
            choices=list(DEPARTMENT_MAP.keys()),
            value="ALL(전체)",
            interactive=True
        )

    gr.Markdown("""
    # 📄 인공지능 PDF Q&A 챗봇
    **여러 PDF 파일을 업로드하고 질문을 입력하면 AI가 답변을 제공합니다!**
    """)

    with gr.Row():
        with gr.Column(scale=1):
            show_files_button = gr.Button("📚 저장된 문서 보기")
            status_output = gr.Textbox(label="📢 상태 메시지")

        with gr.Column(scale=2):
            question_input = gr.Textbox(label="❓ 질문 입력", placeholder="궁금한 내용을 적어주세요.")
            submit_button = gr.Button("🤖 답변 받기")
            answer_output = gr.Textbox(label="📝 AI 답변")

        with gr.Column(scale=3):
            show_history_button = gr.Button("🕘 히스토리 보기")
            history_output = gr.Textbox(label="🗒 전체 대화 내역 | System/Human/AI Message", lines=15, interactive=False)

    # ── 사이드바 이벤트 바인딩 ──
    def create_session():
        sid = f"session_{uuid.uuid4().hex[:8]}"
        init_session(sid)
        rebuild_chain("ALL(전체)")  # 세션마다 RAG 초기화
        return gr.update(choices=list(history_store.keys()), value=sid)

    # 세션 삭제 함수
    def delete_session(session_id: str):
        history_store.pop(session_id, None)

    new_session_btn.click(
        fn=create_session,
        inputs=None,
        outputs=[session_dropdown]
    )

    def remove_session(sid):
        delete_session(sid)
        keys = list(history_store.keys())
        new_val = keys[0] if keys else None
        return gr.update(choices=keys, value=new_val)

    del_session_btn.click(
        fn=remove_session,
        inputs=[session_dropdown],
        outputs=[session_dropdown]
    )

    # ── 부서 선택 시 RAG 체인 재구성 이벤트 바인딩 ──
    def on_department_selected(dept_name):
        rebuild_chain(dept_name)
        return f"✅ '{dept_name}' 부서 기준으로 검색기를 재설정했습니다."

    department_dropdown.change(
        fn=on_department_selected,
        inputs=[department_dropdown],
        outputs=[status_output]
    )

    # ── 파일 업로드 / 질문 / 히스토리 이벤트 ──
    submit_button.click(answer_question, inputs=[question_input, session_dropdown, department_dropdown], outputs=answer_output)
    show_files_button.click(show_stored_documents, outputs=status_output)
    show_history_button.click(show_history, inputs=[session_dropdown], outputs=history_output)

# 앱 실행
demo.launch(debug=True)

In [ ]:
# 드라이브 내 폴더 안에 존재하는 pdf 파일들 저장하는 코드
# 흐름: 폴더 순회 -> pdf/md 파일 1개 찾기 → URL 매칭(유사도 기반) : 파일명을 가지고 URL 알아오기 -> 텍스트 load & split(청킹) → 메타 정보 추가 -> ChromaDB에 저장(자동 임베딩)
# 흐름을 반복하여 처리하는 파이프라인
import os
import re
import unicodedata
import pandas as pd
import logging
import warnings
import sys

from pathlib import Path
from difflib import SequenceMatcher
from io import BytesIO

from langchain_community.document_loaders import PyPDFLoader, TextLoader
# from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

from google.cloud import vision
from pdf2image import convert_from_path

# === Google Vision API 설정 === #
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/Colab Notebooks/lively-sentry-460812-j2-b70abbe49963.json"
vision_client = vision.ImageAnnotatorClient()

# — 설정 — #
ROOT_FOLDER = "/content/drive/MyDrive/National Kongju University Korean Language Representative (국립공주대학교 국문 대표)"   # 순회할 최상위 폴더
PERSIST_DIR = "/content/drive/MyDrive/ChromaDB/knu_chroma_db_all"         # ChromaDB 저장 디렉토리
EMB_MODEL     = "BAAI/bge-m3"
URL_XLSX_PATH = os.path.join(ROOT_FOLDER, "")
SIMILARITY_CUTOFF = 0.7

# PyPDF 경고 무시
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger("pypdf").setLevel(logging.ERROR)

# — 로깅 설정 (Colab에서도 보이게) — #
logger = logging.getLogger()
logger.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)

# 파일명 정규화: NFC → 확장자 제거 → 공백 제거
def normalize_name(name: str) -> str:
    s = unicodedata.normalize("NFC", name)
    s = re.sub(r"\.\w+$", "", s)  # .pdf, .md 등 제거
    return s.replace(" ", "")

# 1) URL 매핑 로드
def load_url_map(path: str) -> dict:
    df = pd.read_excel(path)
    return {
        normalize_name(str(fn)): url
        for fn, url in zip(df["파일명"], df["URL"])
    }
if os.path.isfile(URL_XLSX_PATH):
    URL_MAP = load_url_map(URL_XLSX_PATH)
    logger.info("🔗 URL 매핑 엑셀 로드 완료")
else:
    logger.warning("⚠️ URL 매핑 엑셀 없음")
    URL_MAP = {}

# 2) 가장 유사한 URL 찾아주는 함수
def find_best_url(base_norm: str, url_map: dict, cutoff: float) -> str:
    # 완전 일치
    if base_norm in url_map:
        logger.info(f"🔗 URL 100% 일치: '{base_norm}' → {url_map[base_norm]}")
        return url_map[base_norm]

    # 유사도 탐색
    best_key, best_score = None, 0.0
    for key in url_map:
        score = SequenceMatcher(None, base_norm, key).ratio()
        if score > best_score:
            best_key, best_score = key, score

    if best_score >= cutoff:
        logger.info(f"🔍 유사도({best_score:.2f}) 매칭: '{base_norm}' → '{best_key}' → {url_map[best_key]}")
        return url_map[best_key]
    else:
        logger.info(f"❌ URL 매칭 실패({best_score:.2f}): '{base_norm}' → URL 없음")
        return ""

# 3) Loader 선택 (.pdf/.md)
def get_loader(fp: Path):
    ext = fp.suffix.lower()
    if ext == ".pdf":
        return PyPDFLoader(str(fp))
    if ext == ".md":
        return TextLoader(str(fp), encoding="utf-8")
    return None

# OCR 처리 함수
def ocr_pdf_to_text(pdf_path: str) -> str:
    text_list = []
    try:
        images = convert_from_path(pdf_path)
        for i, image in enumerate(images):
            img_byte_arr = BytesIO()
            image.save(img_byte_arr, format='JPEG')
            content = img_byte_arr.getvalue()
            image_vision = vision.Image(content=content)
            response = vision_client.document_text_detection(image=image_vision)
            text = response.full_text_annotation.text
            logger.info(f"📝 OCR 추출 (p.{i+1}): {len(text)}자")
            text_list.append(text)
    except Exception as e:
        logger.error(f"❌ OCR 처리 실패: {e}")
    return "\n".join(text_list)

# 4) Splitter & Embedding 준비
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
hf_embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL)

# 5) ChromaDB 로드 (폴더가 없으면 내부적으로 새로 생성됨)
db = Chroma(
    persist_directory=PERSIST_DIR,
    embedding_function=hf_embeddings
)
logger.info("🔄 ChromaDB 로드/초기화 완료")

# 6) 파일 순회하며 청킹 + 메타 추가 + DB 저장
for fp in Path(ROOT_FOLDER).rglob("*.*"):
    loader = get_loader(fp)
    if loader is None:
        continue

    base_raw  = fp.name
    base_norm = normalize_name(base_raw)
    # department 추출 (현재 루트 폴더 기준)
    department = Path(ROOT_FOLDER).name

    # 중복 체크: file_name과 department가 모두 일치할 경우에만 중복 간주
    try:
        results = db.get(where={"file_name": {"$eq": base_raw}})
        metadatas = results.get("metadatas", [])
        if any(md.get("department") == department for md in metadatas):
            logger.info(f"⏭️ 이미 저장된 파일: {base_raw} ({department}) → 스킵")
            continue
    except Exception as e:
        logger.warning(f"❗ 중복 검사 실패 (무시하고 진행): {e}")

    logger.info(f"📄 파일 로드 & 청킹 시작: {fp.relative_to(ROOT_FOLDER)}")
    url = find_best_url(base_norm, URL_MAP, SIMILARITY_CUTOFF)

    # 로드 & 청킹
    try:
        chunks = loader.load_and_split(text_splitter=text_splitter)
        logger.info(f"🧩 청크 생성: {len(chunks)}개")
    except Exception as e:
        logger.error(f"❌ '{base_raw}' 로드/청킹 실패: {e}")
        continue

    # 청크가 0개면 사진으로된 pdf문서이므로 OCR 시도
    if not chunks and fp.suffix.lower() == ".pdf":
        logger.warning(f"⚠️ '{base_raw}' 빈 청크 → 사진으로 된 pdf")
        logger.info(f"🖼️ OCR 처리 시도: {base_raw}")
        ocr_text = ocr_pdf_to_text(str(fp))
        if ocr_text.strip():
            chunks = text_splitter.create_documents([ocr_text])
        else:
            logger.warning(f"⚠️ OCR 실패 또는 빈 결과: {base_raw}")
            continue

    # 메타 추가
    for chunk in chunks:
        chunk.metadata.update({
            "file_name":  base_raw,
            "department": department,
            "url":        url
        })

    # DB 저장
    db.add_documents(chunks)
    logger.info(f"➕ '{base_raw}' — {len(chunks)}개 Chroma DB에 저장 완료")

logger.info("🎉 모든 파일 처리 완료 — ChromaDB에 반영되었습니다")
count = db._collection.count()
logger.info(f"현재 DB 청크화된 문서 개수: {count}")

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# — 설정 — #
PERSIST_DIR = "/content/drive/MyDrive/ChromaDB/knu_chroma_db_national_kongju_university_korean_language_representative"
EMB_MODEL   = "BAAI/bge-m3"

# 1) DB 로드
hf_embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL)
db = Chroma(persist_directory=PERSIST_DIR, embedding_function=hf_embeddings)

# 2) 메타정보만 꺼내오기
res = db.get(include=["metadatas"])

# 3) 총 청크 수 및 파일명·부서·URL 출력
metas = res["metadatas"]
print(f"총 청크 수: {len(metas)}\n")
for meta in metas:
    print(
        f"파일명: {meta.get('file_name','Unknown')}  |  "
        f"부서: {meta.get('department','Unknown')}  |  "
        f"URL: {meta.get('url','')}"
    )

In [ ]:
"""
import shutil

PERSIST_DIR = "/content/drive/MyDrive/ChromaDB/knu_chroma_db_smart_infrastructure_engineering"

# 1) 폴더 자체를 날려버리기
shutil.rmtree(PERSIST_DIR)

print(f"🗑️ '{PERSIST_DIR}' 폴더를 완전히 제거했습니다.")
"""

'\nimport shutil\n\nPERSIST_DIR = "/content/drive/MyDrive/ChromaDB/knu_chroma_db_smart_infrastructure_engineering"\n\n# 1) 폴더 자체를 날려버리기\nshutil.rmtree(PERSIST_DIR)\n\nprint(f"🗑️ \'{PERSIST_DIR}\' 폴더를 완전히 제거했습니다.")\n'

In [ ]:
"""
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

PERSIST_DIR = "/content/drive/MyDrive/ChromaDB/knu_chroma_db"
EMB_MODEL   = "BAAI/bge-m3"

# DB 로드
hf_embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL)
db = Chroma(persist_directory=PERSIST_DIR, embedding_function=hf_embeddings)

# 1. where 절로 메타데이터 필터링
try:
    results = db.get(
        where={"department": {"$eq": "Department of Computer Engineering (컴퓨터공학과)"}},
        include=["metadatas", "documents"]  # include에 ids는 쓰면 안 됨
    )
    ids_to_delete = results["ids"]  # 여기서 id가 자동 포함됨

    if ids_to_delete:
        db.delete(ids=ids_to_delete)
        print(f"🗑️ 삭제된 청크 수: {len(ids_to_delete)}")
    else:
        print("✅ 삭제할 청크가 없습니다.")
except Exception as e:
    print(f"❗ 삭제 중 오류 발생: {e}")
"""

In [ ]:
""" OCR 사용 방법 예제
from google.colab import drive
import os
from google.cloud import vision
from pdf2image import convert_from_path
from io import BytesIO

# 서비스 계정 키 경로
# 구글 비전 OCR api 키
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/Colab Notebooks/lively-sentry-460812-j2-b70abbe49963.json"

# Vision API 클라이언트 생성
vision_client = vision.ImageAnnotatorClient()

# OCR 대상 PDF 경로
pdf_path = "/content/drive/MyDrive/Cheonan Campus All Departments (천안캠퍼스 모든 학과)/Software Department (소프트웨어학과)/커뮤니티 (특강공지)/보물의 세계 이벤트.pdf"

# PDF → 이미지로 변환
images = convert_from_path(pdf_path)

# 🚗 6. OCR 처리
for i, image in enumerate(images):
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='JPEG')
    content = img_byte_arr.getvalue()

    image_vision = vision.Image(content=content)
    response = vision_client.document_text_detection(image=image_vision)
    text = response.full_text_annotation.text
    print(f"📝 [페이지 {i+1}] OCR 결과:")
    print(text)
    print("-" * 50)
"""